In [15]:
# ms-python.python added
import os
try:
	os.chdir(os.path.join(os.getcwd(), 'hw2'))
	print(os.getcwd())
except:
	pass

# Homework 2 - Renjie Zhu - A53266114

In [16]:
import scipy as sp
import numpy as np
from sklearn import linear_model
from scipy.io import arff
from io import StringIO

In [17]:
raw, meta = arff.loadarff('5year.arff')

In [18]:
X = raw["class"].astype(int)

In [19]:
A = np.array([(raw["Attr"+str(i+1)]) for i in range(64)]).T
A = np.nan_to_num(A)

In [20]:
model = linear_model.LogisticRegression(C=1.0)
model.fit(A, X)

/home/renjiezhu/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/renjiezhu/.local/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [21]:
pred = model.predict(A)
print(f"Accuracy: {sum(pred==X)/len(X)*100}%.")
# sum(pred==X)

Accuracy: 92.33502538071066%.


In [22]:
def BER(pred, X):
    TP = sum(np.logical_and(pred, X))
    FP = sum(np.logical_and(pred, np.logical_not(X)))
    TN = sum(np.logical_and(np.logical_not(pred), np.logical_not(X)))
    FN = sum(np.logical_and(np.logical_not(pred), X))
    return (0.5 * (FN / (FN + TN) + FP / (FP + TP)))

print(f"BER: {BER(pred, X)*100}%.")


BER: 43.53183667857462%.


In [23]:
AX = np.concatenate((A,X.reshape(-1,1)),axis=1)
np.random.shuffle(AX)

In [24]:
v = len(AX) // 2
t = len(AX) // 4

test_set = AX[:t]
valid_set = AX[t+1:v]
train_set = AX[v+1:]

A = train_set[:,:-1]
X = train_set[:,-1]


In [25]:
model_balanced = linear_model.LogisticRegression(class_weight='balanced')
model_balanced.fit(A, X)

/home/renjiezhu/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/renjiezhu/.local/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [26]:
p_train = model_balanced.predict(A)
p_test = model_balanced.predict(test_set[:,:-1])
p_valid = model_balanced.predict(valid_set[:,:-1])

In [27]:
print(f"Training set:\tAccuracy: {sum(p_train==X)/len(X)*100}%\tBER: {BER(p_train,X)*100}%")
print(f"Test set:\tAccuracy: {sum(p_test==test_set[:,-1])/len(test_set[:,-1])*100}%\tBER: {BER(p_test,test_set[:,-1])*100}%")
print(f"Validation set:\tAccuracy: {sum(p_valid==valid_set[:,-1])/len(valid_set[:,-1])*100}%\tBER: {BER(p_valid,valid_set[:,-1])*100}%")

Training set:	Accuracy: 52.33581584292485%	BER: 45.22291148246035%
Test set:	Accuracy: 52.403520649966154%	BER: 45.99881386492749%
Validation set:	Accuracy: 53.825321597833444%	BER: 46.682712585501825%


In [28]:
# model.fit(A, X)
# p_train = model.predict(A)
# p_test = model.predict(test_set[:,:-1])
# p_valid = model.predict(valid_set[:,:-1])
# print(f"Training set:\tAccuracy: {sum(p_train==X)/len(X)*100}%\tBER: {BER(p_train,X)*100}%")
# print(f"Test set:\tAccuracy: {sum(p_test==test_set[:,-1])/len(test_set[:,-1])*100}%\tBER: {BER(p_test,test_set[:,-1])*100}%")
# print(f"Validation set:\tAccuracy: {sum(p_valid==valid_set[:,-1])/len(valid_set[:,-1])*100}%\tBER: {BER(p_valid,valid_set[:,-1])*100}%")

/home/renjiezhu/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
Training set:	Accuracy: 92.2816519972918%	BER: 38.755393589694926%
Test set:	Accuracy: 92.21394719025051%	BER: 42.840054682159945%
Validation set:	Accuracy: 93.36492890995261%	BER: 46.624743326488705%
/home/renjiezhu/.local/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)